In [1]:
from eunjeon import Mecab
import os
import pandas as pd
from gensim import corpora
import gensim
from gensim.models import CoherenceModel

In [2]:
tagger = Mecab()

In [3]:
path = r'C:\Users\Kislee\PycharmProjects\NLP\판소리춘향'
file_names = [ 
'학술지_춘향_인물_초록_refine.xlsx'
]

In [4]:
file_number = 0

file_name = file_names[file_number]
inputfile = os.path.join(path, file_name)
source_df = pd.read_excel(inputfile, engine="openpyxl")

In [5]:
df = source_df.loc[:, ['저자', '년도', '제목', '초록']]
print("문헌 수 : %d" % len(df))

문헌 수 : 27


In [6]:
attribute = '초록'

In [7]:
df[attribute] = df[attribute].str.replace(pat=r'[^가-힣0-9 ]', repl= r'', regex=True)  # 특수기호 제거
df[attribute] = df[attribute].str.replace(pat=r' +', repl= r' ', regex=True) #연속공백을 공백 하나만 남김

In [8]:
df[attribute][:10]

0    본고는 중국 현대 조선족 여류작가 김인순의 대표작 장편소설 춘향에 관한 작품론적인 ...
1    본고는 여성적 시각에서의 고전읽기가 지니는 역사사회적 방법론의 한계를 극복하면서 음...
2    춘향전은 1920년대까지 가장 많이 팔린 소설책이었다 이러한 춘향전의 인기는 기생인...
3    조선 후기의 사회는 정치적 사회적 문화적 격변을 겪으며 점차 근대의식을 태동하고 있...
4    춘향전에 대한 연구는 풍부하고 다양한 업적을 남기고 있다 그러나 미진한 연구도 없지...
5    본고는 최근 태국의 결혼이주여성들이 많이 증가함에 따라 한국태국 간 다문화 가정 부...
6    본고는 춘향전의 인물 갈등이 춘향과 변학도에만 국한되어 있지 않다는 점과 춘향 이외...
7    한국의 문화원형으로 자리 잡은 춘향 이야기는 여러 가지 예술 형식으로 거듭 재탄생되...
8    경판30장본춘향전과 연극성춘향을 기존의 관습적 분석과 텍스트를 벗어나 새로운 시각으...
9    본 연구는 춘향전의 대표적 이본인 남원고사와 열녀춘향수절가에 나타난 여성 외양에 대...
Name: 초록, dtype: object

In [9]:
abst_tagging = [tagger.pos(x) for x in df[attribute]]

In [10]:
def tag_filtering(word, tag):
    
    if tag.startswith('J') or tag.startswith('ETM') or tag.startswith('EC') or tag.startswith('X'):
        return ''
    
    if tag=='MM' or tag=='NP' or tag=='VV+EC' or tag=='VV+ETM' or tag=='NNB' or tag=='VCP+ETM' or tag == 'MAJ': #MAJ : 및
        return ''
    
    #stopword 필터링 추가 필요한 경우
    #if word in ['춘향','춘향전','인물']: #['연구','중심','방안','고찰']:
    #    return ''
    
    if len(word) <= 1:
        return ''
    
    return word

In [11]:
abst_filtering = [ ' '.join(tag_filtering(word[0],word[1]) for word in abst) for abst in abst_tagging]

In [12]:
tok_absts = [[i for i in doc.split()] for doc in abst_filtering]

In [13]:
word = {}
cnt = 0
for doc in tok_absts:
    for w in doc:
        cnt += 1
        word[w] = word.get(w, 0)+1


print("총 단어수 : %d" % cnt)
print("고유 단어수 : %d" % len(word))

word_freq = []
for key, value in word.items():
    word_freq.append((value, key))

word_freq.sort(reverse=True)
print("상위 단어", word_freq[:100])

총 단어수 : 4286
고유 단어수 : 1364
상위 단어 [(196, '춘향'), (96, '춘향전'), (73, '인물'), (44, '사랑'), (42, '여성'), (42, '서사'), (41, '신분'), (41, '방자'), (38, '기생'), (33, '작품'), (28, '욕망'), (28, '소설'), (28, '도령'), (27, '언술'), (26, '의식'), (23, '이야기'), (22, '주체'), (20, '양상'), (20, '사회'), (20, '문학'), (19, '형상'), (19, '분석'), (19, '구조'), (18, '텍스트'), (18, '변화'), (18, '문제'), (17, '학도'), (17, '의미'), (16, '확인'), (16, '한국'), (16, '자신'), (16, '사령'), (16, '본고'), (16, '등장'), (15, '존재'), (15, '작가'), (15, '동시'), (15, '고전'), (15, '갈등'), (14, '판소리'), (14, '이몽룡'), (14, '연구'), (14, '문화'), (14, '관계'), (13, '현대'), (13, '정절'), (13, '정서'), (13, '유형'), (13, '새로운'), (13, '바탕'), (13, '극복'), (12, '이본'), (12, '원작'), (12, '영화'), (12, '에로스'), (12, '양반'), (12, '애정'), (12, '아니'), (12, '대상'), (12, '대립'), (12, '구성'), (11, '주인공'), (11, '인식'), (11, '연가'), (11, '성취'), (11, '성격'), (11, '방식'), (11, '묘사'), (11, '남성'), (10, '향유'), (10, '하층민'), (10, '지니'), (10, '중국'), (10, '정체'), (10, '인간'), (10, '역할'), (10, '스핀오프'), (10, '비교'), (10, '보이'), (10, '

In [14]:
## 초록에 대한 사전 생성
word_dictionary = corpora.Dictionary(tok_absts)

In [15]:
## corpus matrix
corpus = [word_dictionary.doc2bow(text) for text in tok_absts]

In [17]:
##tf-idf 
from gensim import models
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

In [18]:
print(tok_absts[0])
print(corpus[0])
print(corpus_tfidf[0])

['본고', '중국', '현대', '조선족', '여류', '작가', '김인순', '대표작', '장편', '소설', '춘향', '작품', '입장', '진행', '연구', '한국', '고전', '소설', '춘향전', '병치', '비교', '분석', '방법', '춘향', '이몽룡', '인물', '변용', '분석', '통하', '중국', '현대', '여성', '사랑', '혼인', '작자', '사고', '소설', '춘향', '주제', '의식', '접근', '1970', '출생', '김인순', '중국', '현대', '문단', '영향력', '조선족', '여류', '작가', '70', '출생', '현대', '작가', '1996', '중국', '문단', '데뷔', '2008', '한국', '고전', '소설', '춘향전', '창작', '장편', '소설', '춘향', '발표', '재차', '각광', '동시', '조선족', '신분', '주목', '장편', '소설', '춘향', '2011', '중국', '10', '전국', '소수', '민족', '문학', '창작', '준마', '수상', '한국', '고전', '소설', '춘향전', '인물', '파격', '변형', '소설', '춘향', '단조로운', '중국', '현대', '문단', '타국', '정서', '느낌', '가져다주', '평가', '아니', '소설', '장르', '현대', '변용', '징표', '꼽히', '손색', '중국', '춘향전', '수용', '지평', '춘향', '다원', '문화', '중국', '대륙', '조선족', '제외', '기타', '민족', '한국', '문화', '메시지', '전달', '역할', '담당', '작품', '춘향전', '수용', '차원', '한국', '문화', '전달', '역할', '차원', '모두', '자체', '독보', '문학', '가치', '1970', '1970', '1980', '1990', '1970', '1990', '2003', '2008', '180', '2011']
[(0, 1), (1

In [19]:
#parameters for LDA model
input_num_Topic = 4
input_passes = 2

In [20]:
#generate LDA model
model_list = []
tfidf_model_list = []
for i in range(5):
    model_list.append(gensim.models.ldamodel.LdaModel(corpus, num_topics=input_num_Topic, id2word=word_dictionary, passes=input_passes))
    tfidf_model_list.append(gensim.models.ldamodel.LdaModel(corpus_tfidf, num_topics=input_num_Topic, id2word=word_dictionary, passes=input_passes))

In [21]:
#print perplexity, coherence
print("------------------------normal-------------------------")
for md in model_list:
    cm = CoherenceModel(model=md, corpus=corpus, coherence='u_mass')
    coherence = cm.get_coherence()
    print("Coherence :", coherence)
    print("Perplexity : ", md.log_perplexity(corpus))
    print("-----")
#print("------------------------tf-idf-------------------------")
#for md in tfidf_model_list:
#    cm = CoherenceModel(model=md, corpus=corpus, coherence='u_mass')
#    coherence = cm.get_coherence()
#    print("Coherence :", coherence)
#    print("Perplexity : ", md.log_perplexity(corpus))
#    print("-----")
    

------------------------normal-------------------------
Coherence : -3.799445607393621
Perplexity :  -6.87716595634533
-----
Coherence : -3.57260505743698
Perplexity :  -6.894211728370919
-----
Coherence : -4.9160717719166405
Perplexity :  -6.831965261052493
-----
Coherence : -2.794125512526162
Perplexity :  -6.876386189079708
-----
Coherence : -2.9585032295106544
Perplexity :  -6.861507194778155
-----


In [22]:
#토픽분석 결과 5세트 출력 (상위 10단어)
i=0
for model in model_list:
    i+=1
    print("output", i)
    output_topics = model.print_topics(num_topics=input_num_Topic, num_words=10)
    for topic in output_topics:
        print(topic)
    print()


output 1
(0, '0.021*"춘향" + 0.014*"인물" + 0.011*"춘향전" + 0.010*"도령" + 0.007*"분석" + 0.007*"서사" + 0.006*"관계" + 0.006*"장본" + 0.006*"하층민" + 0.005*"남원"')
(1, '0.041*"춘향" + 0.018*"춘향전" + 0.013*"신분" + 0.012*"기생" + 0.012*"인물" + 0.011*"여성" + 0.011*"사랑" + 0.008*"언술" + 0.008*"작품" + 0.007*"소설"')
(2, '0.023*"방자" + 0.019*"인물" + 0.018*"춘향" + 0.013*"춘향전" + 0.012*"서사" + 0.011*"사령" + 0.009*"이야기" + 0.008*"유형" + 0.006*"여성" + 0.006*"구조"')
(3, '0.045*"춘향" + 0.022*"춘향전" + 0.012*"의식" + 0.010*"욕망" + 0.010*"인물" + 0.008*"주체" + 0.008*"사랑" + 0.008*"텍스트" + 0.006*"문학" + 0.006*"연가"')

output 2
(0, '0.038*"춘향" + 0.022*"춘향전" + 0.015*"욕망" + 0.014*"기생" + 0.011*"소설" + 0.009*"인물" + 0.009*"서사" + 0.008*"신분" + 0.008*"사랑" + 0.007*"여성"')
(1, '0.032*"춘향" + 0.019*"춘향전" + 0.011*"방자" + 0.010*"사랑" + 0.009*"인물" + 0.008*"의식" + 0.008*"여성" + 0.007*"신분" + 0.007*"이야기" + 0.006*"원작"')
(2, '0.023*"춘향" + 0.015*"언술" + 0.013*"인물" + 0.012*"춘향전" + 0.011*"기생" + 0.010*"사랑" + 0.009*"사령" + 0.008*"여성" + 0.008*"도령" + 0.006*"유형"')
(3, '0.044*"춘향" + 0.021*"

In [23]:
#tf-idf 기반 토픽결과 출력 (상위 10단어)
i=0
for model in tfidf_model_list:
    i+=1
    print("output", i)
    output_topics = model.print_topics(num_topics=input_num_Topic, num_words=10)
    for topic in output_topics:
        print(topic)
    print()


output 1
(0, '0.002*"방자" + 0.002*"에로스" + 0.002*"작가" + 0.002*"상편" + 0.002*"하편" + 0.002*"평등" + 0.002*"선택" + 0.001*"몰락" + 0.001*"문제" + 0.001*"고난"')
(1, '0.002*"사령" + 0.002*"태국" + 0.002*"기생" + 0.002*"옥중화" + 0.002*"의식" + 0.001*"1910" + 0.001*"유형" + 0.001*"정담" + 0.001*"교육" + 0.001*"아닌"')
(2, '0.003*"기생" + 0.002*"욕망" + 0.002*"정절" + 0.002*"원작" + 0.002*"여성" + 0.002*"2000" + 0.002*"프랑스" + 0.002*"중국" + 0.002*"스핀오프" + 0.002*"춘향"')
(3, '0.003*"언술" + 0.002*"이몽룡" + 0.002*"분노" + 0.002*"정서" + 0.002*"정체" + 0.002*"고백" + 0.001*"연극" + 0.001*"사랑" + 0.001*"도령" + 0.001*"학도"')

output 2
(0, '0.002*"작가" + 0.002*"2000" + 0.002*"원작" + 0.002*"중국" + 0.002*"방자" + 0.002*"스핀오프" + 0.002*"고사" + 0.002*"연극" + 0.002*"상편" + 0.002*"하편"')
(1, '0.004*"기생" + 0.002*"분노" + 0.002*"프랑스" + 0.002*"옥중화" + 0.002*"갈등" + 0.002*"면모" + 0.002*"의식" + 0.002*"애정" + 0.001*"세계" + 0.001*"1910"')
(2, '0.003*"언술" + 0.003*"욕망" + 0.002*"방자" + 0.002*"극복" + 0.002*"에로스" + 0.002*"텍스트" + 0.002*"서사" + 0.002*"독자" + 0.002*"정절" + 0.002*"이몽룡"')
(3, '0.002*"사령"